#### [Bitcoin API](https://bitcoin.org/en/developer-reference#bitcoin-core-apis)

This notebook takes the [Bitcoin Developers Reference](https://bitcoin.org/en/developer-reference), specifically the [Bitcoin API](https://bitcoin.org/en/developer-reference#bitcoin-core-apis), and demonstrates the use of the Remote Procedure Calls (RPC) via python.  We employ the [regression testing mode](https://bitcoin.org/en/developer-examples#regtest-mode) of bitcoin, regtest, to start with a clean blockchain.

The notebook orders the RPC methods to illustrate how to use the API.  In the [Bitcoin Developers Reference](https://bitcoin.org/en/developer-reference) the RPC methods are in alphabetical order while the python example code below is in *discovery* sequence, starting with creating the genesis block, mining our first bitcoins, sending some to another user and showing updates to the blockchain throughout the process.

The notebook may be useful for beginners looking for a simple python interface to the Bitcoin API; we use the [python bitcoin module](https://github.com/petertodd/python-bitcoinlib) created by Peter Todd for its simplicity.  As well, the value to the user who is new to python is a simple step by step walk thru of the various methods in the API in an incremental introduction without the confusion of thousands of transactions in a typical block on mainnet.

We create two users on the regression test network (regtest):

* Bob  on RPC port 16591 
* Mary on RPC port 16592.  

We create independent peers on the same network.  

Handy shell command to see current nodes:

```ps aux |grep bitcoin```

We could add as many nodes to regtest as we like (upper limit?) and connect them in a round-robin way so that node 1 listens to node 2... and node N listens to node 1.  For our examples here, we use just two nodes in the dictionary ```connects```.

The python script below is designed for Linux (Ubuntu) and would need to be modified for other platforms.


In [ ]:
#!/bin/bash
#regtest_start_network.sh
import os
import shutil

#os.system("killall --regex bitcoin.*")

idir = os.environ['HOME']+'/regtest'
if os.path.isdir(idir):
    shutil.rmtree(idir)

os.mkdir(idir)
connects = {'17591' : '17592', '17592' : '17591'}

for port in connects.keys():
    adir = idir+'/'+port
    os.mkdir(adir)
    args = " -server -listen -port=" + port + \
           " -rpcuser=bitcoinrpc -rpcpassword=P0 -rpcport="+\
           str(int(port)-1000) +\
           " -datadir=" + adir + " -connect=localhost:" + connects[port] +\
           " -regtest -pid="+port+".pid -daemon -debug"
    os.system("bitcoind" + args)


Import the [python bitcoin module](https://github.com/petertodd/python-bitcoinlib) created by Peter Todd.

In [1]:
import bitcoin
import bitcoin.rpc

import bitcoin.core 
import bitcoin.core.script 

#### Connect nodes to bitcoin-rpc module

We hard-code the username and passwords here for pedagogical ease.  We display the blockchain information using the widely used, but deprecated getinfo() RPC.

getinfo() [returns](http://bitcoin.stackexchange.com/questions/18033/what-does-getinfo-output-mean):

version - The version number of this bitcoin-qt or bitcoind program itself. Both of are equivalent. -qt is simply the graphical user interface version

protocolversion: The version of the bitcoin network protocol supported by this client (user agent software).

walletversion: The version of the wallet.dat file. Wallet.dat contains bitcoin addresses and public & private key pairs for these addresses. There is additional data on the wallet. Care must be taken to not restore from an old wallet backup. New addresses generated in the wallet since the old backup was made will not exist in the old backup! [Source](https://en.bitcoin.it/wiki/Wallet)

balance: The total number of bitcoins held in the wallet.dat file.

blocks: The total number of blocks which constitute the shared block chain.

timeoffset: Seconds of difference between this node's "wall clock time" and the median reported by our network peers.

connections: the number of peers on the bitcoin P2P network that this node is connected to.

proxy: If using a proxy to connect to the network, listed here, otherwise blank.

difficulty: the current mining difficulty factor. Difficulty is increased as more miners and more hash compute power compete to be the next one to have a block of transactions added to the blockchain.

testnet: Boolean value (true OR false). There is a parallel bitcoin network, the testnet, where trials and experiments may be carried out without impacting the official, live bitcoin P2P network

keypoololdest: timestamp (UNIX epoch) of the oldest key in the keypool

keypoolsize: A number of addresses are kept in reserve by the client. This is the size of that reserve.

paytxfee: Specifies what fee the client is willing to pay to expedite transactions. Miners may choose to ignore transactions that do not pay a fee, and these fee-less transactions will have low priority on queue of pending transaction and may linger there for hours.

errors: This field may inform of different status conditions. Full list of error codes in source file [bitcoinrpc.h](https://dev.visucore.com/bitcoin/doxygen/bitcoinrpc_8h_source.html) (Examples: "Bitcoin not connected", "database error", "Keypool ran out"...) 


In [2]:
bitcoin.SelectParams('regtest')

Bob  = bitcoin.rpc.RawProxy("http://bitcoinrpc:P0@127.0.0.1:16591")
Mary = bitcoin.rpc.RawProxy("http://bitcoinrpc:P0@127.0.0.1:16592")

info = Bob.getinfo()
for key in info.keys():
    print key + ' : ' + str(info[key])

connections : 2
errors : This is a pre-release test build - use at your own risk - do not use for mining or merchant applications
blocks : 0
paytxfee : 0E-8
keypoololdest : 1442863295
walletversion : 60000
difficulty : 0E-8
testnet : False
version : 119900
proxy : 
protocolversion : 70002
timeoffset : 0
balance : 0E-8
relayfee : 0.00001000
keypoolsize : 101


#### Network attributes
The following RCPs are used to determine the network attributes.  We highlight the informational RPCs and leave the [AddNode](https://bitcoin.org/en/developer-reference#addnode) and [GetAddedNodeInfo](https://bitcoin.org/en/developer-reference#getaddednodeinfo) for later investigation.

#### Network RPCs

* [AddNode](https://bitcoin.org/en/developer-reference#addnode): attempts to add or remove a node from the addnode list, or to try a connection to a node once.
* [GetAddedNodeInfo](https://bitcoin.org/en/developer-reference#getaddednodeinfo): returns information about the given added node, or all added nodes (except onetry nodes). Only nodes which have been manually added using the addnode RPC will have their information displayed.
* [GetConnectionCount](https://bitcoin.org/en/developer-reference#GetConnectionCount): returns the number of connections to other nodes.
* [GetNetTotals](https://bitcoin.org/en/developer-reference#GetNetTotals): returns information about network traffic, including bytes in, bytes out, and the current time.
* [GetNetworkInfo](https://bitcoin.org/en/developer-reference#GetNetworkInfo): returns information about the node’s connection to the network. New in 0.9.2, Updated in 0.10.0
* [GetPeerInfo](https://bitcoin.org/en/developer-reference#GetPeerInfo): returns data about each connected network node. Updated in 0.10.0
* [Ping](https://bitcoin.org/en/developer-reference#Ping): sends a P2P ping message to all connected nodes to measure ping time. Results are provided by the getpeerinfo RPC pingtime and pingwait fields as decimal seconds. The P2P ping message is handled in a queue with all other commands, so it measures processing backlog, not just network ping.

In [3]:
getnetworkinfo = Bob.getnetworkinfo()
print '\ngetnetworkinfo\n'
print getnetworkinfo
getpeerinfo = Bob.getpeerinfo()
print '\ngetpeerinfo\n'
print getpeerinfo
getconnectioncount = Bob.getconnectioncount()
print '\ngetconnectioncount\n'
print getconnectioncount
getnettotals = Bob.getnettotals()
print '\ngetnettotals\n'
print getnettotals



getnetworkinfo

{u'localservices': u'0000000000000001', u'warnings': u'This is a pre-release test build - use at your own risk - do not use for mining or merchant applications', u'connections': 2, u'subversion': u'/Satoshi:0.11.99/', u'version': 119900, u'relayfee': Decimal('0.00001000'), u'protocolversion': 70002, u'timeoffset': 0, u'localaddresses': [], u'networks': [{u'limited': False, u'proxy_randomize_credentials': False, u'reachable': False, u'name': u'ipv4', u'proxy': u''}, {u'limited': False, u'proxy_randomize_credentials': False, u'reachable': False, u'name': u'ipv6', u'proxy': u''}, {u'limited': False, u'proxy_randomize_credentials': False, u'reachable': False, u'name': u'onion', u'proxy': u''}]}

getpeerinfo

[{u'inflight': [], u'lastsend': 1442863535, u'banscore': 0, u'addr': u'127.0.0.1:43759', u'whitelisted': False, u'inbound': True, u'version': 70002, u'synced_blocks': -1, u'subver': u'/Satoshi:0.11.99/', u'lastrecv': 1442863535, u'timeoffset': 0, u'pingtime': Decimal('

#### Block Chain RPCs

* [GetBestBlockHash](https://bitcoin.org/en/developer-reference#getbestblockhash): returns the header hash of the most recent block on the best block chain. New in 0.9.0
* [GetBlock](https://bitcoin.org/en/developer-reference#GetBlock): gets a block with a particular header hash from the local block database either as a JSON object or as a serialized block.
* [GetBlockChainInfo](https://bitcoin.org/en/developer-reference#GetBlockChainInfo): provides information about the current state of the block chain. New in 0.9.2, Updated in 0.10.0
* [GetBlockCount](https://bitcoin.org/en/developer-reference#GetBlockCount): returns the number of blocks in the local best block chain.
* [GetBlockHash](https://bitcoin.org/en/developer-reference#GetBlockHash): returns the header hash of a block at the given height in the local best block chain.
* [GetChainTips](https://bitcoin.org/en/developer-reference#GetChainTips): returns information about the highest-height block (tip) of each local block chain. New in 0.10.0
* [GetDifficulty](https://bitcoin.org/en/developer-reference#GetDifficulty): returns the proof-of-work difficulty as a multiple of the minimum difficulty.
* [GetMemPoolInfo](https://bitcoin.org/en/developer-reference#GetMemPoolInfo): returns information about the node’s current transaction memory pool. New in 0.10.0
* [GetRawMemPool](https://bitcoin.org/en/developer-reference#GetRawMemPool): returns all transaction identifiers (TXIDs) in the memory pool as a JSON array, or detailed information about each transaction in the memory pool as a JSON object.
* [GetTxOut](https://bitcoin.org/en/developer-reference#GetTxOut): returns details about a transaction output. Only unspent transaction outputs (UTXOs) are guaranteed to be available.
* [GetTxOutProof](https://bitcoin.org/en/developer-reference#GetTxOutProof): returns a hex-encoded proof that one or more specified transactions were included in a block. New in 0.11.0
* [GetTxOutSetInfo](https://bitcoin.org/en/developer-reference#GetTxOutSetInfo): returns statistics about the confirmed unspent transaction output (UTXO) set. Note that this call may take some time and that it only counts outputs from confirmed transactions—it does not count outputs from the memory pool.
* [VerifyChain](https://bitcoin.org/en/developer-reference#VerifyChain): verifies each entry in the local block chain database.
* [VerifyTxOutProof](https://bitcoin.org/en/developer-reference#VerifyTxOutProof): verifies that a proof points to one or more transactions in a block, returning the transactions the proof commits to and throwing an RPC error if the block is not in our best block chain. New in 0.11.0


In [4]:
blockchaininfo = Bob.getblockchaininfo()
getblockcount  =  Bob.getblockcount()
getbestblockhash =  Bob.getbestblockhash()
getdifficulty  =  Bob.getdifficulty()
getchaintips   =  Bob.getchaintips()
getmempoolinfo = Bob.getmempoolinfo()

print '\nblockchaininfo\n'
print blockchaininfo
print '\ngetblockcount ' + str(getblockcount)
print '\ngetbestblockhash\n'
print getbestblockhash
print '\ngetdifficulty ' + str(getdifficulty)
print '\ngetchaintips\n'
print getchaintips
print '\ngetmempoolinfo\n'
print getmempoolinfo

print '\n\n'

bestblockhash  =  blockchaininfo['bestblockhash']
blocks         =  blockchaininfo['blocks']
print '\nblocks = ' + str(blocks)
print '\nbestblockhash = ' + str(bestblockhash)




blockchaininfo

{u'pruned': False, u'blocks': 0, u'chainwork': u'0000000000000000000000000000000000000000000000000000000000000002', u'chain': u'regtest', u'difficulty': Decimal('0E-8'), u'headers': 0, u'verificationprogress': Decimal('1.00000000'), u'bestblockhash': u'0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206'}

getblockcount 0

getbestblockhash

0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206

getdifficulty 0E-8

getchaintips

[{u'status': u'active', u'branchlen': 0, u'hash': u'0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206', u'height': 0}]

getmempoolinfo

{u'bytes': 0, u'size': 0}




blocks = 0

bestblockhash = 0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206


#### Bootstrap

Bootstrap some bitcoins so we can spend them.  In regtest, you use generate and mine 101 blocks rewarding you with 50 bitcoins in a coinbase transaction.  On testnet, you use a faucet to get some bitcoins.  On mainnet, you need actual bitcoins.  We will stick with regtest for now.

In [5]:
## N.B. our balance is zero in the genesis block
print 'Initial balance, before any mining ' + str(Bob.getbalance())

Bob.generate(101)

print 'Balance after mining 101 blocks ' + str(Bob.getbalance())


Initial balance, before any mining 0E-8
Balance after mining 101 blocks 50.00000000


#### Raw Transaction RPCs

* [CreateRawTransaction](https://bitcoin.org/en/developer-reference#createrawtransaction): creates an unsigned serialized transaction that spends a previous output to a new output with a P2PKH or P2SH address. The transaction is not stored in the wallet or transmitted to the network.
* [DecodeRawTransaction](https://bitcoin.org/en/developer-reference#DecodeRawTransaction): decodes a serialized transaction hex string into a JSON object describing the transaction.
* [DecodeScript](https://bitcoin.org/en/developer-reference#DecodeScript): decodes a hex-encoded P2SH redeem script.
* [GetRawTransaction](https://bitcoin.org/en/developer-reference#GetRawTransaction): gets a hex-encoded serialized transaction or a JSON object describing the transaction. By default, Bitcoin Core only stores complete transaction data for UTXOs and your own transactions, so the RPC may fail on historic transactions unless you use the non-default txindex=1 in your Bitcoin Core startup settings.
* [SendRawTransaction](https://bitcoin.org/en/developer-reference#SendRawTransaction): validates a transaction and broadcasts it to the peer-to-peer network.
* [SignRawTransaction](https://bitcoin.org/en/developer-reference#SignRawTransaction): signs a transaction in the serialized transaction format using private keys stored in the wallet or provided in the call.

#### Generate some bitcoins with a mining start
Now let's look at the block structure, rerunning code from above.  Note that the block height is now 101.  There is one transaction for 50BTC.  Note that this is a coinbase transaction with just a vout and no vin.

How to make serialized transaction?

In [6]:
getblockcount = Bob.getblockcount()
print '\ngetblockcount = ' + str(getblockcount)

getblockhash = Bob.getblockhash(getblockcount)
print '\ngetblockhash = ' + str(getblockhash)

print '\ngetblock\n'
getblock = Bob.getblock(getblockhash)
print getblock

tx = getblock['tx']
print '\n' + str(len(tx)) + ' Transactions\n'
print tx

for i in range(len(tx)):
    print '\nSerialized Transaction #' + str(i) +'\n'
    serializedTX = Bob.getrawtransaction(tx[i],0)
    print serializedTX
    print '\nRaw Transaction\n'
    rawTX = Bob.getrawtransaction(tx[i],1)
    print rawTX
    print '\nDecoded Transaction\n'
    decodedTX    = Bob.decoderawtransaction(serializedTX)
    print decodedTX



getblockcount = 101

getblockhash = 33b8a212a21785d2685be2ba16ed4c28f35fae16ccb5a03433180a3c99731db8

getblock

{u'merkleroot': u'13b0ec1286d11d52bbe5123a0f63e8b9c93f420e17755a36dbc7c1b1995a602a', u'nonce': 1, u'previousblockhash': u'5ddab7b07126746e3476f7443428682be23e206d72073ba2d89e311eb9d0c2c5', u'hash': u'33b8a212a21785d2685be2ba16ed4c28f35fae16ccb5a03433180a3c99731db8', u'version': 3, u'tx': [u'13b0ec1286d11d52bbe5123a0f63e8b9c93f420e17755a36dbc7c1b1995a602a'], u'chainwork': u'00000000000000000000000000000000000000000000000000000000000000cc', u'height': 101, u'difficulty': Decimal('0E-8'), u'confirmations': 1, u'time': 1442863626, u'bits': u'207fffff', u'size': 180}

1 Transactions

[u'13b0ec1286d11d52bbe5123a0f63e8b9c93f420e17755a36dbc7c1b1995a602a']

Serialized Transaction #0

01000000010000000000000000000000000000000000000000000000000000000000000000ffffffff0401650101ffffffff0100f2052a010000002321021808f691f52aba529fc1bb37bb465522480a226fcb9a555f42185f1263d2f303ac00000000

Raw

Now that we have a raw transaction, let's look at the details.  As noted, the first 100 blocks in regtest are blank, necessary for mining our first coinbase reward of 50BTC.  This shows in block 101 with a single transaction, denoted rawTX below.  rawTX is a JSON object with:

1. blockhash from block 101
2. vout an array (len=1) of scriptPubKey, a JSON object of reqSigs and a value (50BTC here)
3. vin an array (len=1) with a key called "coinbase", some value and a sequence number

We veirfy that the serialized transaction hex is the same as the "hex" entry in the transaction.  We note that the txid is repeated in the raw transaction.  We also note that the confirmations = 1 (no further mining has been done on this block), that there is a single transaction (the mining of 50btc) and the [blocktime](https://en.bitcoin.it/wiki/Block_timestamp) and time are the same, while the locktime is the beginning of the epoch.  

Note that an address has been created, in general an array of addresses, each corresponding to a transaction output (vout).  Here, a new address has been created corresponding to the miner's address for the 50BTC reward.

In [7]:
from datetime import datetime as dt
#import pytz

print rawTX 
print '\n\n'

print 'blockhash = ' + str(rawTX['blockhash']) + '\n'
for i in range(len(rawTX['vout'])):
    spk = rawTX['vout'][i]['scriptPubKey']
    print 'vout ' + str(i) + ' : ' + str(spk) + '\n'
    for field in spk.keys(): 
        #['reqSigs','hex','addresses','asm','type']:
        print 'vout ' + str(i) + ' ' + field + '  : ' + str(spk[field])
    print 'vout ' + str(i) + ' value : ' + str(rawTX['vout'][i]['value'])
    print 'vout ' + str(i) + ' n     : ' + str(rawTX['vout'][i]['n'])

print '\nserialized hex = ' + str(rawTX['hex'])
print 'Is serialized hex == rawTX["hex"]? ' + str(rawTX['hex']==serializedTX) + '\n'

for i in range(len(rawTX['vin'])):
    spk = rawTX['vin'][i]
    print 'vin ' + str(i) + ' : ' + str(spk) + '\n'
    for field in spk.keys(): 
        #['reqSigs','hex','addresses','asm','type']:
        print 'vin ' + str(i) + ' ' + field + '  : ' + str(spk[field])

print '\n'
for field in ['txid','blocktime','version','confirmations','time','locktime']:
    if field in ['blocktime','time','locktime']:
        print field + ' = ' + str(rawTX[field]) +\
        ' ' + dt.fromtimestamp(rawTX[field]).strftime('%Y-%m-%d:%H%M%S')
    else:
        print field + ' = ' + str(rawTX[field])


{u'blockhash': u'33b8a212a21785d2685be2ba16ed4c28f35fae16ccb5a03433180a3c99731db8', u'vout': [{u'scriptPubKey': {u'reqSigs': 1, u'hex': u'21021808f691f52aba529fc1bb37bb465522480a226fcb9a555f42185f1263d2f303ac', u'addresses': [u'mzfBBHNVGSqKChj2g43YVUyUg1dw7d7xza'], u'asm': u'021808f691f52aba529fc1bb37bb465522480a226fcb9a555f42185f1263d2f303 OP_CHECKSIG', u'type': u'pubkey'}, u'value': Decimal('50.00000000'), u'n': 0}], u'hex': u'01000000010000000000000000000000000000000000000000000000000000000000000000ffffffff0401650101ffffffff0100f2052a010000002321021808f691f52aba529fc1bb37bb465522480a226fcb9a555f42185f1263d2f303ac00000000', u'vin': [{u'coinbase': u'01650101', u'sequence': 4294967295}], u'txid': u'13b0ec1286d11d52bbe5123a0f63e8b9c93f420e17755a36dbc7c1b1995a602a', u'blocktime': 1442863626, u'version': 1, u'confirmations': 1, u'time': 1442863626, u'locktime': 0}



blockhash = 33b8a212a21785d2685be2ba16ed4c28f35fae16ccb5a03433180a3c99731db8

vout 0 : {u'reqSigs': 1, u'hex': u'21021808f6

In [ ]:
##The P2PKH or P2SH addresses used in this transaction, 
##or the computed P2PKH address of any pubkeys in this transaction. 

##N.B. the developer's guide states that the address is a "A P2PKH or P2SH address"
##but we see this initiate transactions as a type="pubkey".  Should check on mainnet.

addresses = rawTX['vout'][0]['scriptPubKey']['addresses']
print addresses
for address in addresses:
    print address
    print Bob.getbalance()
    print Bob.getreceivedbyaddress(address,0)
print Bob.listreceivedbyaddress(0)

Let's create an address and send some bitcoins there.  Some things to notice:

0.  We set up a new proxy, using a different RPC port on regtest
1.  Address is *NOT* on blockchain, it is just a 32 byte hash that is almost certainly unique and never before used.
2.  We send some coins to new address, note the transaction structure
3.  The mempool shows the pending transaction
4.  We need to mine to write the transaction to the blockchain

In [8]:
print 'Mary\'s balance = ' + str(Mary.getbalance())
#print 'Mary\'s peers: '
#print Mary.getpeerinfo()

getnewaddress = Mary.getnewaddress()
print '\nNew address ' + str(getnewaddress)
print '\nMary\'s address has received how many BTC? ' +\
str(Mary.getreceivedbyaddress(getnewaddress,0))

##have Bob (proxy) send 25 bitcoins to Mary
txid = Bob.sendtoaddress(getnewaddress,25)


Mary's balance = 0E-8

New address mqcNX8muHm8GAKVphaECWzLhLPLWNa1S7H

Mary's address has received how many BTC? 0E-8


In [9]:
getmempoolinfo = Bob.getmempoolinfo()
getrawmempool = Bob.getrawmempool(True)
print '\ngetmempoolinfo ' + str(getmempoolinfo)
print '\ngetrawmempool'
print getrawmempool
print '\n'
for key in getrawmempool.keys():
    for field in getrawmempool[key].keys():
        print str(field) + ' : ' + str(getrawmempool[key][field])
#print '\ntxid from sendtoaddress output ' + str(txid)
print '\nIs the send txid the same as memory pool txid? ****' +\
    str(txid == getrawmempool.keys()[0]) + '****'

print '\nMary\'s balance before mining = ' + str(Mary.getbalance())
print 'Bob\'s balance before mining = ' + str(Bob.getbalance())

##how can I see transaction details before mining?
print '\nMemory Pool Raw Transaction Data\n'
import pprint
pprint.pprint(Bob.getrawtransaction(txid,1))

##N.B. no transaction on the blockchain yet!!!



getmempoolinfo {u'bytes': 192, u'size': 1}

getrawmempool
{u'eaeca12dc604e3bf4d1ad45a014435d3b0dea00da5307a0fa1ede901837ca978': {u'fee': Decimal('0.00000192'), u'startingpriority': Decimal('6410256410.25641060'), u'height': 101, u'depends': [], u'time': 1442863696, u'currentpriority': Decimal('6410256410.25641060'), u'size': 192}}


fee : 0.00000192
startingpriority : 6410256410.25641060
height : 101
depends : []
time : 1442863696
currentpriority : 6410256410.25641060
size : 192

Is the send txid the same as memory pool txid? ****True****

Mary's balance before mining = 0E-8
Bob's balance before mining = 24.99999808

Memory Pool Raw Transaction Data

{u'hex': u'0100000001a98337e984f85b1a078a140fd0ba6a6b942db101946550bf2df510dd6c597e090000000049483045022100f817d8412b41ca8f23a482d57822534d4264fefff80fa7bb657923ffcb1a0007022025c039a23dcc75681d1cb49e9e2c958433ef936e8e339c10daabdb48ffc54cbe01feffffff0200f90295000000001976a9146eb78f62da0f091f92ff6d4de3ff8d7cc03ec43288ac40f80295000000001976a

Bob mines 6 more blocks, after the fifth, the 25BTC sent to Mary is confirmed and shows in her balance.  Note that the transaction created in block 101, when Bob sent Mary 25BTC shows up in the first block mined, but the balance isn't updated until some number of blocks have been processed.  How is this determined????

In [10]:
for i in range(7):
    Bob.generate(1)
    getblockcount   =  Bob.getblockcount()
    getblockhash   =  Bob.getblockhash(getblockcount)
    getblock       =  Bob.getblock(getblockhash)
    print 'Block #' + str(getblockcount) + ' Mary\'s balance ' + str(Mary.getbalance())
    print 'txids ' + str(getblock['tx'])
    

Block #102 Mary's balance 0E-8
txids [u'268d1466634e2d10b5e1dfc2b178b9d84170b64f5b6ec98cc82f595887ce83a9', u'eaeca12dc604e3bf4d1ad45a014435d3b0dea00da5307a0fa1ede901837ca978']
Block #103 Mary's balance 0E-8
txids [u'a49437d5754af4729c06e3cd48e4c1d82c7b982b880bc33a8a8dae3fa84fe0dc']
Block #104 Mary's balance 0E-8
txids [u'082160b61e5ea6c8ab2a32ed9d38cf401a04c06c5c27f1ae9c7afaccb7cc110d']
Block #105 Mary's balance 0E-8
txids [u'51cc64bb0294dbbc34e4c5dec062d80e0ffb1dfe780339e988917eb4a3063c07']
Block #106 Mary's balance 0E-8
txids [u'4d31ed0d601d12050a5d3c8094e2429b12fe0d7a623de599d7a5065ab3d165b8']
Block #107 Mary's balance 0E-8
txids [u'66c0d019dc4dbeaa7db385abdc5d6be62458c85f543610a68d27672af82e663a']
Block #108 Mary's balance 0E-8
txids [u'20bbfc2911e28d3f9c21e8d2b448995b94146ea7605e1c77fc6aa05d1a861520']


In [11]:
print ' Mary\'s balance ' + str(Mary.getbalance())

 Mary's balance 25.00000000


This code determines whether a transaction is included in a Merkle Tree.  From [Luke Dashjr](https://github.com/Crypto-Expert/stratum-mining/blob/master/lib/merkletree.py)

In [12]:
# Eloipool - Python Bitcoin pool server
# Copyright (C) 2011-2012  Luke Dashjr <luke-jr+eloipool@utopios.org>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as
# published by the Free Software Foundation, either version 3 of the
# License, or (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.
#
# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

# original code https://github.com/Crypto-Expert/stratum-mining/blob/master/lib/merkletree.py
# http://code.runnable.com/U3jqtyYUmAUxtsSS/bitcoin-block-merkle-root-python

from hashlib import sha256
from util import doublesha

class MerkleTree:
    def __init__(self, data, detailed=False):
        self.data = data
        self.recalculate(detailed)
        self._hash_steps = None
    
    def recalculate(self, detailed=False):
        L = self.data
        steps = []
        if detailed:
            detail = []
            PreL = []
            StartL = 0
        else:
            detail = None
            PreL = [None]
            StartL = 2
        Ll = len(L)
        if detailed or Ll > 1:
            while True:
                if detailed:
                    detail += L
                if Ll == 1:
                    break
                steps.append(L[1])
                if Ll % 2:
                    L += [L[-1]]
                L = PreL + [doublesha(L[i] + L[i + 1]) for i in range(StartL, Ll, 2)]
                Ll = len(L)
        self._steps = steps
        self.detail = detail
    
    def hash_steps(self):
        if self._hash_steps == None:
            self._hash_steps = doublesha(''.join(self._steps))
        return self._hash_steps
        
    def withFirst(self, f):
        steps = self._steps
        for s in steps:
            f = doublesha(f + s)
        return f
    
    def merkleRoot(self):
        return self.withFirst(self.data[0])

# MerkleTree tests
def _test():
    import binascii
    import time    
        
    mt = MerkleTree([None] + [binascii.unhexlify(a) for a in [
        '999d2c8bb6bda0bf784d9ebeb631d711dbbbfe1bc006ea13d6ad0d6a2649a971',
        '3f92594d5a3d7b4df29d7dd7c46a0dac39a96e751ba0fc9bab5435ea5e22a19d',
        'a5633f03855f541d8e60a6340fc491d49709dc821f3acb571956a856637adcb6',
        '28d97c850eaf917a4c76c02474b05b70a197eaefb468d21c22ed110afe8ec9e0',
    ]])
    assert(
        b'82293f182d5db07d08acf334a5a907012bbb9990851557ac0ec028116081bd5a' ==
        binascii.b2a_hex(mt.withFirst(binascii.unhexlify('d43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7')))
    )
    
    print '82293f182d5db07d08acf334a5a907012bbb9990851557ac0ec028116081bd5a'
    txes = [binascii.unhexlify(a) for a in [
        'd43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7',
        '999d2c8bb6bda0bf784d9ebeb631d711dbbbfe1bc006ea13d6ad0d6a2649a971',
        '3f92594d5a3d7b4df29d7dd7c46a0dac39a96e751ba0fc9bab5435ea5e22a19d',
        'a5633f03855f541d8e60a6340fc491d49709dc821f3acb571956a856637adcb6',
        '28d97c850eaf917a4c76c02474b05b70a197eaefb468d21c22ed110afe8ec9e0',
    ]]
             
    s = time.time()
    mt = MerkleTree(txes)
    for x in range(100):
        y = int('d43b669fb42cfa84695b844c0402d410213faa4f3e66cb7248f688ff19d5e5f7', 16)
        #y += x
        coinbasehash = binascii.unhexlify("%x" % y)
        x = binascii.b2a_hex(mt.withFirst(coinbasehash))

    print x
    print time.time() - s

if __name__ == '__main__':
    _test()
    

999d2c8bb6bda0bf784d9ebeb631d711dbbbfe1bc006ea13d6ad0d6a2649a971
a2bdc4dc9f21ca0147a13855c513fe404875d4d6eb57c80f80950ae842baf3b5
90f12ff4be0ac8744d0478146f8e9951afcdeba3597834091ca01df41f7e2b7d


This code snippet illustrates a simplified proof-of-work algorithm *not* used by miners; by incementing the nonce and trying difficulties from 1-31 bits (2 - 2^32).  Note that this runs for a long time, more than 20 minutes on a 4GB-RAM Ubuntu box.

From [Mastering Bitcoin](http://chimera.labs.oreilly.com/books/1234000001802/ch08.html#_proof_of_work_algorithm)

In [ ]:
#!/usr/bin/env python
# example of proof-of-work algorithm

import hashlib
import time

max_nonce = 2 ** 32 # 4 billion

def proof_of_work(header, difficulty_bits):

    # calculate the difficulty target
    target = 2 ** (256-difficulty_bits)

    for nonce in xrange(max_nonce):
        hash_result = hashlib.sha256(str(header)+str(nonce)).hexdigest()

        # check if this is a valid result, below the target
        if long(hash_result, 16) < target:
            print "Success with nonce %d" % nonce
            print "Hash is %s" % hash_result
            return (hash_result,nonce)

    print "Failed after %d (max_nonce) tries" % nonce
    return nonce


if __name__ == '__main__':

    nonce = 0
    hash_result = ''

    # difficulty from 0 to 31 bits
    for difficulty_bits in xrange(32):

        difficulty = 2 ** difficulty_bits
        print "Difficulty: %ld (%d bits)" % (difficulty, difficulty_bits)

        print "Starting search..."

        # checkpoint the current time
        start_time = time.time()

        # make a new block which includes the hash from the previous block
        # we fake a block of transactions - just a string
        new_block = 'test block with transactions' + hash_result

        # find a valid nonce for the new block
        (hash_result, nonce) = proof_of_work(new_block, difficulty_bits)

        # checkpoint how long it took to find a result
        end_time = time.time()

        elapsed_time = end_time - start_time
        print "Elapsed Time: %.4f seconds" % elapsed_time

        if elapsed_time > 0:

            # estimate the hashes per second
            hash_power = float(long(nonce)/elapsed_time)
            print "Hashing Power: %ld hashes per second" % hash_power

Difficulty: 1 (0 bits)
Starting search...
Success with nonce 0
Hash is ff8253ed10b5f719d52a709a66af8cd5e2054f702e675af4ca0cae70f0988634
Elapsed Time: 0.0001 seconds
Hashing Power: 0 hashes per second
Difficulty: 2 (1 bits)
Starting search...
Success with nonce 0
Hash is 22c608547e239faf5c353e7ebd204042760b93891d1d0be9ab488d36c73c077b
Elapsed Time: 0.0001 seconds
Hashing Power: 0 hashes per second
Difficulty: 4 (2 bits)
Starting search...
Success with nonce 2
Hash is 0635f41cdb98c6e73516f84fc88da19a13a3bac6298dbfc0df5170bac93ba4dd
Elapsed Time: 0.0001 seconds
Hashing Power: 21732 hashes per second
Difficulty: 8 (3 bits)
Starting search...
Success with nonce 9
Hash is 1c1c105e65b47142f028a8f93ddf3dabb9260491bc64474738133ce5256cb3c1
Elapsed Time: 0.0001 seconds
Hashing Power: 68759 hashes per second
Difficulty: 16 (4 bits)
Starting search...
Success with nonce 25
Hash is 0f7becfd3bcd1a82e06663c97176add89e7cae0268de46f94e7e11bc3863e148
Elapsed Time: 0.0002 seconds
Hashing Power: 119156 has